In [11]:
# Importação de bibliotecas necessárias
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import html
import requests
import time
import re

def search_google_scholar(keyword):
    url = f"https://scholar.google.com/scholar?q={keyword}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find('div', {'id': 'gs_ab_md'})
    if results:
        return results.text.split()[1]
    else:
        return "Data not available"

def search_pubmed(keyword):
    url = f"https://pubmed.ncbi.nlm.nih.gov/?term={keyword}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find('div', {'class': 'results-amount'})
    if results:
        return results.text.strip().split()[0]
    else:
        return "Data not available"

def search_science_direct_selenium(keyword):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)
    
    url = f"https://www.sciencedirect.com/search?qs={keyword}&articleTypes=REV%2CFLA&lastSelectedFacet=articleTypes"
    driver.get(url)
    
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.search-body-results-text"))
        )
        text = element.text
        results = int(re.search(r'\d+', text).group())
        return str(results)
    except Exception as e:
        return f"Exception encountered: {e}"
    finally:
        driver.quit()


    
if __name__ == "__main__":
    keywords = ["DoWhy", "CausalML"]

    for keyword in keywords:
        print(f"Fetching data for {keyword}...")
        
        # Google Scholar
        gs_count = search_google_scholar(keyword)
        print(f"Number of articles mentioning {keyword} in Google Scholar: {gs_count:5}")

        # PubMed
        pm_count = search_pubmed(keyword)
        print(f"Number of articles mentioning {keyword} in         PubMed: {pm_count:5}")

        # Science Direct
        sd_count = search_science_direct(keyword)
        print(f"Number of articles mentioning {keyword} in Science Direct: {sd_count:5}")

        print("-" * 75)
        
        # Adding sleep to avoid rate-limiting
        time.sleep(5)

Fetching data for DoWhy...
Number of articles mentioning DoWhy in Google Scholar: 633  
Number of articles mentioning DoWhy in         PubMed: 14   
Number of articles mentioning DoWhy in Science Direct: Failed to retrieve data, HTTP Status Code: 403
---------------------------------------------------------------------------
Fetching data for CausalML...
Number of articles mentioning CausalML in Google Scholar: 504  
Number of articles mentioning CausalML in         PubMed: 10,630,105
Number of articles mentioning CausalML in Science Direct: Failed to retrieve data, HTTP Status Code: 403
---------------------------------------------------------------------------


In [8]:
url = f"https://www.sciencedirect.com/search?qs={keyword}&articleTypes=REV%2CFLA&lastSelectedFacet=articleTypes"
url

'https://www.sciencedirect.com/search?qs=CausalML&articleTypes=REV%2CFLA&lastSelectedFacet=articleTypes'

In [9]:
from bs4 import BeautifulSoup
import requests
import re

keyword='DoWhy'
url = f"https://www.sciencedirect.com/search?qs={keyword}&articleTypes=REV%2CFLA&lastSelectedFacet=articleTypes"
response = requests.get(url)
response

<Response [403]>

In [ ]:

soup = BeautifulSoup(response.content, 'html.parser')

# Use the class name to locate the element
element = soup.find('span', {'class': 'search-body-results-text'})

# Proceed if the element exists
if element:
    text = element.get_text()
    results = int(re.search(r'\d+', text).group())
    return str(results)
else:
    return "Data not available"
